In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))
    if len(filenames) > 3:
        print("...")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/captcha-hacker/sample_submission.csv
/kaggle/input/captcha-hacker/test/task1/x4LPcV5n6IvLj4vz.png
/kaggle/input/captcha-hacker/test/task1/W88fVMlAs5IpsXn4.png
/kaggle/input/captcha-hacker/test/task1/ZWDL6pUMfPu5c9jh.png
...
/kaggle/input/captcha-hacker/test/task2/jMalnsI5a5IWxYAi.png
/kaggle/input/captcha-hacker/test/task2/ihE9HHgyOINGEMcO.png
/kaggle/input/captcha-hacker/test/task2/ZATEVW3P5s0akZjd.png
...
/kaggle/input/captcha-hacker/test/task3/cXBlxYfvQWbiK7dn.png
/kaggle/input/captcha-hacker/test/task3/5gEp1jR9jNNfuqlk.png
/kaggle/input/captcha-hacker/test/task3/hEQ0WQtB9B7j8C2f.png
...
/kaggle/input/captcha-hacker/train/annotations.csv
/kaggle/input/captcha-hacker/train/task1/H85RQ6dbWUvLSIDV.png
/kaggle/input/captcha-hacker/train/task1/n2GC8uY1N4QfvVxe.png
/kaggle/input/captcha-hacker/train/task1/XOqfRx2R6SnoEjFr.png
...
/kaggle/input/captcha-hacker/train/task2/Mr4B2zxXk92hyzn9.png
/kaggle/input/captcha-hacker/train/task2/SIuRCnlK8VS91FhX.png
/kaggle/input/captcha-h

In [ ]:
import csv
import cv2
import numpy as np
import random
import os

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models

In [ ]:
TRAIN_PATH = "/kaggle/input/captcha-hacker/train"
TEST_PATH = "/kaggle/input/captcha-hacker/test"
device = "cuda"
# try device = "cuda" 
# and change your settings/accelerator to GPU if you want it to run faster

In [ ]:
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
result = ['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [ ]:
class Task1Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task1")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
        img = cv2.resize(img, (224, 224))
#         img = np.mean(img, axis=2)
#         print(img.shape)

        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
            return torch.FloatTensor(img / 256), int(label)

    def __len__(self):
        return len(self.data)

In [ ]:
class Task2Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task2")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        
        labels = [0 for i in range(72)]

        if label[0].isalpha(): 
            idx = alphabet.index(label[0]) + 10
            labels[idx] = 1
        else:
            labels[int(label[0])] = 1

        if len(label) > 1:
            if label[1].isalpha():
                idx = alphabet.index(label[1]) + 10
                labels[idx + 36] = 1
            else:
                labels[int(label[1]) + 36] = 1
                 
        img = cv2.imread(f"{self.root}/{filename}")
        img = cv2.resize(img, (224, 224))
        
        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
                return torch.FloatTensor(img / 256), torch.tensor(labels)

    def __len__(self):
        return len(self.data)

In [ ]:
class Task3Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task3")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        
        labels = [0 for i in range(144)]
        
        if label[0].isalpha(): 
            idx = alphabet.index(label[0]) + 10
            labels[idx] = 1
        else:
            labels[int(label[0])] = 1

        if len(label) > 1:
            if label[1].isalpha():
                idx = alphabet.index(label[1]) + 10
                labels[idx + 36] = 1
            else:
                labels[int(label[1]) + 36] = 1
    
        if len(label) > 2:
            if label[2].isalpha():
                idx = alphabet.index(label[2]) + 10
                labels[idx + 72] = 1
            else:
                labels[int(label[2]) + 72] = 1
            
        if len(label) > 3:
            if label[3].isalpha():
                idx = alphabet.index(label[3]) + 10
                labels[idx + 108] = 1
            else:
                labels[int(label[3]) + 108] = 1
            
        img = cv2.imread(f"{self.root}/{filename}")
        img = cv2.resize(img, (480, 360))
        
        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
                return torch.FloatTensor(img / 256), torch.tensor(labels)

    def __len__(self):
        return len(self.data)

In [ ]:
class Model1(nn.Module):
    def __init__(self):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(1024, 512),
#             nn.LeakyReLU(),
#             nn.Linear(512, 10)
#         )

        super().__init__()
        # 載入 ResNet18 類神經網路結構
        self.model = models.resnet18(pretrained=True)
        # 修改輸出層輸出數量
        self.model.fc = nn.Linear(512, 10)
        
        
    def forward(self, x):
#         b, h, w = x.shape
#         x = x.view(b, h*w)
#         return self.layers(x)
        logits = self.model(x)
        return logits

In [ ]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 72)  # 72
        
    def forward(self, x):
        logits = self.model(x)
        return logits

In [ ]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 144)  # 144
        
    def forward(self, x):
        logits = self.model(x)
        return logits

In [ ]:
train_data = []
val_data = []

file = open(f'{TRAIN_PATH}/annotations.csv', newline='')
with file as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        if random.random() < 0.9:
            train_data.append(row)
        else:
            val_data.append(row)

# task1
train1_ds = Task1Dataset(train_data, root=TRAIN_PATH)
train1_dl = DataLoader(train1_ds, batch_size=64, num_workers=4, drop_last=True, shuffle=True)

val1_ds = Task1Dataset(val_data, root=TRAIN_PATH)
val1_dl = DataLoader(val1_ds, batch_size=64, num_workers=4, drop_last=False, shuffle=False)

# task2
train2_ds = Task2Dataset(train_data, root=TRAIN_PATH)
train2_dl = DataLoader(train2_ds, batch_size=64, num_workers=4, drop_last=True, shuffle=True)

val2_ds = Task2Dataset(val_data, root=TRAIN_PATH)
val2_dl = DataLoader(val2_ds, batch_size=64, num_workers=4, drop_last=False, shuffle=False)

#task3
train3_ds = Task3Dataset(train_data, root=TRAIN_PATH)
train3_dl = DataLoader(train3_ds, batch_size=64, num_workers=4, drop_last=True, shuffle=True)

val3_ds = Task3Dataset(val_data, root=TRAIN_PATH)
val3_dl = DataLoader(val3_ds, batch_size=64, num_workers=4, drop_last=False, shuffle=False)

In [ ]:
model1 = Model1().to(device)
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()


for epoch in tqdm(range(20)):
    # train
    model1.train()
    for image, label in train1_dl:
        image = image.to(device)
        label = label.to(device)
#         print(image.shape)
        image = image.permute(0,3,1,2)

        pred = model1(image)
        loss = loss_fn(pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # test
    if (epoch+1) % 10 == 0:
        sample_count = 0
        correct_count = 0
        model1.eval()
        for image, label in val1_dl:
            image = image.to(device)
            label = label.to(device)
            image = image.permute(0,3,1,2)

            pred = model1(image)
            loss = loss_fn(pred, label)

            pred = torch.argmax(pred, dim=1)

            sample_count += len(image)
            correct_count += (label == pred).sum()
        print(f"Epoch [{epoch+1}]")
        print("accuracy (validation):", correct_count / sample_count)
# [batch, channels, height, width]

 50%|█████     | 10/20 [02:29<02:31, 15.14s/it]

Epoch [10]
accuracy (validation): tensor(0.9952, device='cuda:0')


100%|██████████| 20/20 [04:58<00:00, 14.92s/it]

Epoch [20]
accuracy (validation): tensor(0.9952, device='cuda:0')


In [ ]:
model2 = Model2().to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-3)
loss_fn = nn.MultiLabelSoftMarginLoss()

for epoch in tqdm(range(30)):
    # train
    model2.train()
    for image, label in train2_dl:
        image = image.to(device)
        label = label.to(device)

        image = image.permute(0,3,1,2)
        
        pred = model2(image)
        loss = loss_fn(pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # test
    if (epoch+1) % 10 == 0: 
        sample_count = 0
        correct_count = 0
        model2.eval()
        for image, label in val2_dl:
            image = image.to(device)
            label = label.to(device)

            image = image.permute(0,3,1,2)

            pred = model2(image)
            loss = loss_fn(pred, label)

    #         print("pred")
            pred1, pred2 = torch.split(pred, 36, dim=1)
            pred1 = torch.argmax(pred1, dim=1)
            pred2 = torch.argmax(pred2, dim=1)
    #         print(pred1.shape, pred2.shape)
    #         print("label")
            label1, label2 = torch.split(label, 36, dim=1)
            label1 = torch.argmax(label1, dim=1)
            label2 = torch.argmax(label2, dim=1)
    #         print(label1.shape, label2.shape)

            sample_count += len(image)
            for i in range(len(pred1)):
                if pred1[i] == label1[i] and pred2[i] == label2[i]:
                    correct_count += 1
    #         correct_count += (label == pred).sum()
        print(f"Epoch [{epoch+1}]")
        print("accuracy (validation):", correct_count / sample_count)

 33%|███▎      | 10/30 [03:00<06:05, 18.26s/it]

Epoch [10]
accuracy (validation): 1.0


 67%|██████▋   | 20/30 [06:01<03:02, 18.29s/it]

Epoch [20]
accuracy (validation): 1.0


100%|██████████| 30/30 [09:02<00:00, 18.07s/it]

Epoch [30]
accuracy (validation): 1.0


In [ ]:
model3 = Model3().to(device)
optimizer = torch.optim.Adam(model3.parameters(), lr=1e-3)
loss_fn = nn.MultiLabelSoftMarginLoss()

for epoch in tqdm(range(50)):
    # train
    model3.train()
    for image, label in train3_dl:
        image = image.to(device)
        label = label.to(device)

        image = image.permute(0,3,1,2)
        
        pred = model3(image)
        loss = loss_fn(pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        # test
        sample_count = 0
        correct_count = 0
        model3.eval()
        for image, label in val3_dl:
            image = image.to(device)
            label = label.to(device)

            image = image.permute(0,3,1,2)

            pred = model3(image)
            loss = loss_fn(pred, label)

            pred1, pred2, pred3, pred4 = torch.split(pred, 36, dim=1)
            pred1 = torch.argmax(pred1, dim=1)
            pred2 = torch.argmax(pred2, dim=1)
            pred3 = torch.argmax(pred3, dim=1)
            pred4 = torch.argmax(pred4, dim=1)
    #         print(pred1.shape, pred2.shape, pred3.shape, pred4.shape)
            label1, label2, label3, label4 = torch.split(label, 36, dim=1)
            label1 = torch.argmax(label1, dim=1)
            label2 = torch.argmax(label2, dim=1)
            label3 = torch.argmax(label3, dim=1)
            label4 = torch.argmax(label4, dim=1)
    #         print(label1.shape, label2.shape, label3.shape, label4.shape)

            sample_count += len(image)
            for i in range(len(pred1)):
                if pred1[i] == label1[i] and pred2[i] == label2[i] and pred3[i] == label3[i] and pred4[i] == label4[i]:
                    correct_count += 1
    #         correct_count += (label == pred).sum()
    
        print(f"Epoch [{epoch+1}]")
        print("accuracy (validation):", correct_count / sample_count)

 20%|██        | 10/50 [14:00<56:33, 84.83s/it] 

Epoch [10]
accuracy (validation): 0.963076923076923


 40%|████      | 20/50 [28:01<42:22, 84.76s/it]

Epoch [20]
accuracy (validation): 0.9938461538461538


 60%|██████    | 30/50 [42:02<28:16, 84.84s/it]

Epoch [30]
accuracy (validation): 0.9969230769230769


 80%|████████  | 40/50 [56:03<14:08, 84.84s/it]

Epoch [40]
accuracy (validation): 1.0


100%|██████████| 50/50 [1:10:05<00:00, 84.11s/it]

Epoch [50]
accuracy (validation): 0.9876923076923076


In [ ]:
torch.save(model1.state_dict(), 'save1.pt')
torch.save(model2.state_dict(), 'save2.pt')
torch.save(model3.state_dict(), 'save3.pt')